
<a href="https://colab.research.google.com/github/kokchun/Databehandling-AI22/blob/main/Lectures/L3-merge.ipynb" target="_parent"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> &nbsp; for interacting with the code

---
# Lecture notes - Merge, join, concatenate, append, read_html
---

This is the lecture note for **data wrangling** - but it's built upon contents from pandas and previous course: 
- Python programming

<p class = "alert alert-info" role="alert"><b>Note</b> that this lecture note gives a brief introduction to merging. I encourage you to read further about merging.

Read more 
- [Merge, join, concatenate ...](https://pandas.pydata.org/docs/user_guide/merging.html)

- [read_html()](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html?highlight=read_html#pandas.read_html)

- [append()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.append.html)

---

## Setup
- creating 3 DataFrames

In [1]:
import pandas as pd 
import numpy as np 

df1 = pd.DataFrame(np.arange(16).reshape(4,4), columns=list("ABCD"))
df2 = pd.DataFrame(np.zeros((3,4)), columns=list("ABCD"))
df3 = pd.DataFrame(np.ones((4,3)), index=[1,5,10,15])
df1

,A,B,C,D
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [2]:
df2

,A,B,C,D
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


In [3]:
df3

,0,1,2
1,1.0,1.0,1.0
5,1.0,1.0,1.0
10,1.0,1.0,1.0
15,1.0,1.0,1.0


## Concat

- concatenates pandas objects along particular axis
- there are optional set logic along other axes
- concat() copies the data, if using concat() on many datasets, use list comprehensions

Set logic: 

<img src="../assets/outer-inner-join.png" width=500/>


In [4]:
# same column names, just add the second df rows to the first
pd.concat([df1,df2]) # by default axis = 0 or "rows"

,A,B,C,D
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0
3,12.0,13.0,14.0,15.0
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


In [5]:
# concats on the index 
# NOTE: NaNs due to df2 shape is (3,4) while df1 shape is (4,4)
pd.concat([df1,df2], axis = "columns")

,A,B,C,D,A,B,C,D
0,0,1,2,3,0.0,0.0,0.0,0.0
1,4,5,6,7,0.0,0.0,0.0,0.0
2,8,9,10,11,0.0,0.0,0.0,0.0
3,12,13,14,15,NaN,NaN,NaN,NaN


In [6]:
# NOTE: index 1 is shared while others are not 
pd.concat([df1,df3], axis = 1)  # join="outer by default"

,A,B,C,D,0,1,2
0,0.0,1.0,2.0,3.0,NaN,NaN,NaN
1,4.0,5.0,6.0,7.0,1.0,1.0,1.0
2,8.0,9.0,10.0,11.0,NaN,NaN,NaN
3,12.0,13.0,14.0,15.0,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1.0,1.0,1.0
10,NaN,NaN,NaN,NaN,1.0,1.0,1.0
15,NaN,NaN,NaN,NaN,1.0,1.0,1.0


In [7]:
# inner join - set logic intersection
pd.concat([df1,df3], axis = 1, join="inner") 

,A,B,C,D,0,1,2
1,4,5,6,7,1.0,1.0,1.0


## Append

Specific case of concat
- concatenates along axis 0 
- join = outer
- can't choose other axis or joins
- doesn't modify original df unlike in append for lists

In [8]:
df1.append(df2, ignore_index=True) # simple 

/var/folders/3s/7mxc35_16j71s8tnrxvhfg4m0000gn/T/ipykernel_71852/852695297.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(df2, ignore_index=True) # simple


,A,B,C,D
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0
3,12.0,13.0,14.0,15.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0


## Merge

- high performance join operations similar to relational databases
- performance is faster using merge/join in Pandas than in SQL

Relational algebra
- one-to-one - joining 2 dfs on their index
- many-to-one - joining a unique index to ≥ 1 cols in different df
- many-to-many - joining columns on columns

In [9]:
# merge
left = pd.DataFrame({"key": ["K0", "K0", "K1", "K2"],
                     "A": ["A0", "A1", "A2", "A3"],
                     "B": ["B0", "B1", "B2", "B3"],
                     })
right = pd.DataFrame({"key": ["K0", "K1", "K2", "K3"],
                     "C": ["C0", "C1", "C2", "C3"],
                      "D": ["D0", "D1", "D2", "D3"]})
print(left)
print(right)


  key   A   B
0  K0  A0  B0
1  K0  A1  B1
2  K1  A2  B2
3  K2  A3  B3
  key   C   D
0  K0  C0  D0
1  K1  C1  D1
2  K2  C2  D2
3  K3  C3  D3


In [10]:
pd.merge(left, right, on="key", indicator=True) # how= "inner" default


,key,A,B,C,D,_merge
0,K0,A0,B0,C0,D0,both
1,K0,A1,B1,C0,D0,both
2,K1,A2,B2,C1,D1,both
3,K2,A3,B3,C2,D2,both


In [11]:
left.merge(right, on=["key"], how="outer", indicator=True)

,key,A,B,C,D,_merge
0,K0,A0,B0,C0,D0,both
1,K0,A1,B1,C0,D0,both
2,K1,A2,B2,C1,D1,both
3,K2,A3,B3,C2,D2,both
4,K3,NaN,NaN,C3,D3,right_only


In [12]:
left.merge(right, on="key", how="left", indicator=True)

,key,A,B,C,D,_merge
0,K0,A0,B0,C0,D0,both
1,K0,A1,B1,C0,D0,both
2,K1,A2,B2,C1,D1,both
3,K2,A3,B3,C2,D2,both


In [13]:
left = pd.DataFrame(
    {
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

right = pd.DataFrame(
    {
        "key1": ["K0", "K1", "K1", "K2"],
        "key2": ["K0", "K0", "K0", "K0"],
        "A": ["A0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

print(left)

print(right)

# only keys present in left
left.merge(right, on=["key1", "key2"], how="left", indicator=True)

  key1 key2   A   B
0   K0   K0  A0  B0
1   K0   K1  A1  B1
2   K1   K0  A2  B2
3   K2   K1  A3  B3
  key1 key2   A   D
0   K0   K0  A0  D0
1   K1   K0  C1  D1
2   K1   K0  C2  D2
3   K2   K0  C3  D3


,key1,key2,A_x,B,A_y,D,_merge
0,K0,K0,A0,B0,A0,D0,both
1,K0,K1,A1,B1,NaN,NaN,left_only
2,K1,K0,A2,B2,C1,D1,both
3,K1,K0,A2,B2,C2,D2,both
4,K2,K1,A3,B3,NaN,NaN,left_only


In [14]:
# keys in right
left.merge(right, on=["key1", "key2"], how="right", indicator=True, suffixes=["_left","_right"])

,key1,key2,A_left,B,A_right,D,_merge
0,K0,K0,A0,B0,A0,D0,both
1,K1,K0,A2,B2,C1,D1,both
2,K1,K0,A2,B2,C2,D2,both
3,K2,K0,NaN,NaN,C3,D3,right_only


## Join
- combine cols of two potentially different index dfs

In [15]:
left = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]}, index=["K0", "K1", "K2"]
)


right = pd.DataFrame(
    {"C": ["C0", "C2", "C3"], "D": ["D0", "D2", "D3"]}, index=["K0", "K2", "K3"]
)

left.join(right) # joins on left

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


## Webscrape HTML tables 

- [solve SSLCertVerificationError on mac](https://stackoverflow.com/questions/68275857/urllib-error-urlerror-urlopen-error-ssl-certificate-verify-failed-certifica)




In [16]:
tables_list = pd.read_html(
    "https://en.wikipedia.org/wiki/List_of_potentially_habitable_exoplanets")

tables_list.__len__()

7

In [17]:
df = tables_list[0] # first table in the html page 
df.head()

,Object,Star,Star type,Mass (M⊕),Radius (R⊕),Density (g/cm3),Flux (F⊕),Teq (K),Period (days),Distance (ly),Refs/Notes
0,Earth,Sun (Sol),G2V,1.00,1.00,5.514,1.00,255,365.25,0.00,[3]Reported for reference. Only planet known t...
1,Teegarden's Star b,Teegarden's Star,M7V,≥1.05,~1.02,—,1.15,264,4.91,12.58,[5]Radius is estimated[6]
2,TOI 700 d,TOI 700,M2V,~1.72,1.14,—,0.87,246,37.40,101.00,NaN
3,Kepler-1649c,Kepler-1649,M5V,—,1.06,—,0.75,237,19.50,301.00,[7]
4,TRAPPIST-1d,TRAPPIST-1,M8V,0.30,0.78,3.39,1.04,258,4.05,39.00,Confirmed to be rocky[8][9]


In [18]:
fifa_tables = pd.read_html("https://en.wikipedia.org/wiki/FIFA_World_Cup", match="Highest attendances")

fifa_tables[0].head()

Year        Hosts Venues/ Cities Total attendance † Matches  \
   Year        Hosts Venues/ Cities Total attendance † Matches   
0  1930      Uruguay            3/1             590549      18   
1  1934        Italy            8/8             363000      17   
2  1938       France           10/9             375700      18   
3  1950       Brazil            6/6            1045246      22   
4  1954  Switzerland            6/6             768607      26   

  Avg. attendance Highest attendances ‡                                    \
  Avg. attendance                Number                             Venue   
0           32808                 93000    Estadio Centenario, Montevideo   
1           21353                 55000        Stadio Nazionale PNF, Rome   
2           20872                 58455      Olympique de Colombes, Paris   
3           47511           173,850[86]  Maracanã Stadium, Rio de Janeiro   
4           29562                 63000            Wankdorf Stadium, Bern   

                                       
                              Game(s)  
0  Uruguay 6–1 Yugoslavia, Semi-final  
1     Italy 2–1 Czechoslovakia, Final  
2     France 1–3 Italy, Quarter-final  
3  Brazil 1–2 Uruguay, Deciding match  
4     West Germany 3–2 Hungary, Final

---

Kokchun Giang

[LinkedIn][linkedIn_kokchun]

[GitHub portfolio][github_portfolio]

[linkedIn_kokchun]: https://www.linkedin.com/in/kokchungiang/
[github_portfolio]: https://github.com/kokchun/Portfolio-Kokchun-Giang

---